In [1]:
import json, glob, os, numpy
from data_process_functions import *
from sklearn.ensemble import ExtraTreesClassifier 
os.chdir('/Users/adamyedidia/breast_cancer/spreadsheets/')
import openpyxl, pprint


SHEET_NAME = 'manisha_new.xlsx'
#LIST_OF_BINARY_COLS = ['Q', 'R', 'S', 'T','U','V','W','X','Y','Z','AA','AB','AC']
LIST_OF_BINARY_COLS = ['D','F','G','H','I','J','K','L','M','T','U','W','X','Y','AA','AE','AH','AJ']
LIST_OF_REAL_COLS = ['A','C','N','O','P','Q','R','S','V','Z','U','V','Z']  
ANSWER_COL = 'AQ'
        
with open('image-features-InceptionV3-CC.json') as data_file:
     image_features_map = json.load(data_file)
        
data,feature_names,result=dataprocess(SHEET_NAME,LIST_OF_BINARY_COLS,LIST_OF_REAL_COLS,ANSWER_COL)
yy=[1 if x=='M' else 0 for x in result]
print 'cancers numbers: %d' %yy.count(1)
y=np.array(yy)
print 'num data points:', len(yy)
X_structure=data[:,1:]
feature_names_structure=feature_names[1:]
n=X_structure.shape[0]





Real-valued Features:
ID
AGE
WEIGHT
HEIGHT
BIRTH
AGEPREG
MAAGE
MPAGE
DENSITY
PRIORBX
ASSESS
DENSITY
PRIORBX

Binary Features:
RACE
LOC
SPROCS
FBC
PBC
POC
SMOKE
ALCOHOL
ASHKENAZI
HORMONE
ASSESS
FIRSTMAM
FROMSCREEN
CLINFIND
FINDTYPE
SIDE
BXTYPE
BXPATH_CONCAT

Result:
SXRESULT
data size is: 1119 X 157
test reasult is: SXRESULT
cancers numbers: 135
num data points: 1118


In [2]:
pass

In [3]:
pass

In [4]:
f_structure= ExtraTreesClassifier(n_estimators=200,max_depth=12,max_features=80,class_weight = {1:7},
                              random_state=123)
f_report = ExtraTreesClassifier(n_estimators=200,max_depth=10,max_features=500,class_weight = {1:5},
                              random_state=123)
'''
feature_ranking(f_structure,feature_names_structure,X_structure,y,'importantfeature_structure.txt',pp=1)
feature_ranking(f_report,feature_names_report_prune,ft_report_prune,y,'importantfeature_report.txt',pp=1)
'''

"\nfeature_ranking(f_structure,feature_names_structure,X_structure,y,'importantfeature_structure.txt',pp=1)\nfeature_ranking(f_report,feature_names_report_prune,ft_report_prune,y,'importantfeature_report.txt',pp=1)\n"

In [ ]:
from sklearn.cross_validation import StratifiedShuffleSplit

n_str=100
n_report=180
weight =3
random_state= 127

#sss = StratifiedShuffleSplit(y, 10, test_size=0.05, random_state=1237)
#rs = cross_validation.ShuffleSplit(len(y), n_iter=10,test_size=.1, random_state=1237)

from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(y, 10, shuffle=True, random_state=random_state)

k=0

YTEST=[]
YTEST_index=[]
Probability=[]

for train_index, test_index in skf: 
        ff = ExtraTreesClassifier(n_estimators=200,max_depth=10,max_features=100
                                      ,class_weight = {1:weight},random_state=123)
        
        print 'new sets'
        X_train_struct, X_test_struct = X_structure[train_index], X_structure[test_index]
        #X_train_txt, X_test_txt = ft_report_prune[train_index], ft_report_prune[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        YTEST.append(y_test)
        YTEST_index.append(test_index)
   
        #k+=1
        #feature_ranking(f_structure,feature_names_structure,X_train_struct,y_train,'importantfeature_strcture_'+str(k)+'.txt',pp=0)
        #feature_ranking(f_report,feature_names_report_prune,X_train_txt,y_train,'importantfeature_report_'+str(k)+'.txt',pp=0)

        indx_structure=get_sorted_ft_idx(f_structure,X_train_struct,y_train,n=n_str)
        #indx_txt=get_sorted_ft_idx(f_report,X_train_txt,y_train,n=n_report)
        
        #X_train=np.concatenate((X_train_struct[:,indx_structure], X_train_txt[:,indx_txt]), axis=1)
        #X_test=np.concatenate((X_test_struct[:,indx_structure], X_test_txt[:,indx_txt]), axis=1)
        X_train = X_train_struct
        X_test = X_test_struct
        
       
        ff.fit(X_train, y_train)

        ff_proba=ff.predict_proba(X_test)
        Probability.append(ff_proba)
        indx1=np.where(y_test==1)
        indx2=np.where(y_test==0)
        yy= np.random.random_sample((len(y_test),))
        plt.clf()
        #plt.scatter([0, 1], [0, 1])
        plt.scatter(ff_proba[:,1][indx1], yy[indx1], c=y_test[indx1], s=50,marker='o')
        plt.scatter(ff_proba[:,1][indx2], yy[indx2], c=y_test[indx2], s=50,marker='+',label='no')
        plt.xlabel('probability')
        plt.ylabel('Random number')
        plt.legend(loc='best')
        plt.show()
        #prb=cus_probability(threshold, ff_proba[:,1])
        #prb=np.array(prb)

new sets
new sets
new sets
new sets
new sets
new sets
new sets
new sets
new sets
new sets

In [6]:
p_all=[]
f_all=[]
r_all=[]
roc_all=[]
patients=[]
for ii in range(len(YTEST)):
    y_test=YTEST[ii]
    ff_proba=Probability[ii]
    roc=[]
    p=[]
    r=[]
    f=[]
    #results=[]
    pat=[]
    Thresh=[]
    for i in range(30):
        threshold=float(float(i+1)/80)
        Thresh.append(threshold)
        predicted=cus_predict(threshold, ff_proba[:,1])
        #results.append(predicted)

        fpr_rf, tpr_rf, _ = roc_curve(y_test,predicted)
        roc.append([fpr_rf, tpr_rf])
        p.append(precision_score(y_test,predicted))
        r.append(recall_score(y_test,predicted))
        f.append(f1_score(y_test,predicted))
        print(classification_report(y_test,predicted) )
        print 'roc_auc is %f :' %roc_auc_score(y_test,predicted)
        pt=len(predicted)-np.count_nonzero(predicted)
        print 'non cancer patient : %d' %pt
        pat.append(pt)

    #plt.scatter(prb[:,1],y_test)

    '''
    indxFb=np.intersect1d(np.where(ff_proba[:,1]>0.7 ),np.where(y_test==0))
    indxFm=np.intersect1d(np.where(ff_proba[:,1]<0.3),np.where(y_test==1))
    fp= X_test[indxFm][:,0]  ### The first column is the patient ID
    fn=X_test[indxFb][:,0]
    falsepositive.extend(fp)
    falsenegative.extend(fn)
    '''
    p_all.append(p)
    r_all.append(r)
    f_all.append(f)
    roc_all.append(roc)
    patients.append(pat)

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        67
          1       0.11      1.00      0.19         8

avg / total       0.01      0.11      0.02        75

roc_auc is 0.500000 :
non cancer patient : 0
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        67
          1       0.11      1.00      0.19         8

avg / total       0.01      0.11      0.02        75

roc_auc is 0.500000 :
non cancer patient : 0
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        67
          1       0.11      1.00      0.19         8

avg / total       0.01      0.11      0.02        75

roc_auc is 0.500000 :
non cancer patient : 0
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        67
          1       0.09      0.88      0.17         8

avg / total       0.01      0.09      0.02        75

roc_a

/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)



roc_auc is 0.559701 :
non cancer patient : 8
             precision    recall  f1-score   support

          0       1.00      0.18      0.30        67
          1       0.13      1.00      0.23         8

avg / total       0.91      0.27      0.30        75

roc_auc is 0.589552 :
non cancer patient : 12
             precision    recall  f1-score   support

          0       0.94      0.24      0.38        67
          1       0.12      0.88      0.21         8

avg / total       0.85      0.31      0.36        75

roc_auc is 0.556903 :
non cancer patient : 17
             precision    recall  f1-score   support

          0       0.96      0.33      0.49        67
          1       0.13      0.88      0.23         8

avg / total       0.87      0.39      0.46        75

roc_auc is 0.601679 :
non cancer patient : 23
             precision    recall  f1-score   support

          0       0.91      0.43      0.59        67
          1       0.12      0.62      0.20         8

avg / tota

/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
n=2
#m=f_all.shape[1]
m=10

f_all=np.array(f_all)
r_all=np.array(r_all)
patients=np.array(patients)
print 'trainging size: %d' %f_all[:-n,i].shape
for i in range(m):
    print average(f_all[:-n,i])
print "  \n test result"
for i in range(m):
    print average(f_all[-n:,i])
print " \n train result"
for i in range(m):
    print average(r_all[:-n,i])
print "  \ntest result"

for i in range(m):
    print average(r_all[-n:,i])

print "\n train result"
t=sum([len(l) for l in YTEST_index[:-n] ])
for i in range(m):
    print float(sum(patients[:-n,i])/float(sum([len(l) for l in YTEST_index[:-n] ])))

print ' \n test result'
for i in range(m):
    print float(sum(patients[-n:,i])/float(sum([len(l) for l in YTEST_index[-n:] ])))
    
patients[9,3]

In [ ]:
#[54486, 86819,111803]
TestID=54486
newdata=[]
newtxt=[]
ID=[]
newy=[]
for i in range(data.shape[0]):
    if data[i,0] ==TestID:
        ID.append(data[i,0])
        newdata.append(data[i,1:])
        newtxt.append(ft_report_p[i,:])
        newy.append(y[i])
    '''
    else:
        if yy[i]==1:
            ID.append(data[i,0])
            newdata.append(data[i])
            newtxt.append(ft_report_p[i,:])
            newy.append(yy[i])
    '''
newdata=np.array(newdata)
newtxt=np.array(newtxt)
newy=np.array(newy)

In [ ]:
X_test_special=np.concatenate((newdata[:,indx_structure], newtxt[:,indx_txt]), axis=1)
ff_proba=ff.predict_proba(X_test_special)
Probability.append(ff_proba)
indx1=np.where(newy==1)
indx2=np.where(newy==0)
yy= np.random.random_sample((len(newy),))
plt.clf()
#plt.scatter([0, 1], [0, 1])
plt.scatter(ff_proba[:,1][indx1], yy[indx1], c=newy[indx1], s=50,marker='o',label='cancer')
plt.scatter(ff_proba[:,1][indx2], yy[indx2], c=newy[indx2], s=50,marker='+',label='no cancer')
plt.xlabel('probability')
plt.ylabel('Random number')
plt.legend(loc='best')
plt.show()
#prb=cus_probability(threshold, ff_proba[:,1])
#prb=np.array(prb)

In [ ]:
print str(X_test.shape)
print len( indx_structure)
print len( indx_txt)